# Data Preparation

In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from matplotlib import image
from tqdm import tqdm
#from PIL import Image

# Variables to process data
df = pd.read_csv('/kaggle/input/pokemon-images-and-types/pokemon.csv')
X = []
m = len(df['Name'])
Y = []
list_element_count = 0

# Loader to keep track of how many images have been processed in the dataset
def how_many_loaded(currentImgNo, totalLenOfImages):
    x = currentImgNo / totalLenOfImages * 100
    print (str(x) + ' % processed')
    
# Process data => turn images into arrays

def img_to_lst(img_location):
    '''
    Parameters:
    img_location -- location of image
    
    Retuns:
    img_list -- list of image pixels
    '''
    
    img = image.imread(img_location)
    
    pixel_val_list = []
    
    # Get red pixel values
    for y in img:
        for x in y:
            pixel_val_list.append(x[0])
            
    # Get green pixel values
    for y in img:
        for x in y:
            pixel_val_list.append(x[1])
            
    # Get blue pixel values
    for y in img:
        for x in y:
            pixel_val_list.append(x[2])
                
    return np.array(pixel_val_list)
    print 


# Remove jpg's from the img dataset + create new processed image array + create true values array
for x in tqdm(os.listdir('/kaggle/input/pokemon-images-and-types/images/images')):
    temp = None
    temp = x.split('.')
    if temp[1] == 'png':
        X.append(img_to_lst('/kaggle/input/pokemon-images-and-types/images/images/'+x))
        if df.iloc[list_element_count]['Type1'] == 'Fire':
            Y.append(1)
        else:
            Y.append(0)
    list_element_count += 1
#     how_many_loaded(list_element_count, m)

# Further processing data to transform it into => (no. of inputs X no. of examples) format
X = np.array(X)
Y = np.array(Y)
X = np.squeeze(X)
X = X.T
Y = Y.T
Y = Y.reshape(Y.shape[0], 1)
dim = X.shape[0]

print ('Y', Y.shape)
print ('X', X.shape)

100%|██████████| 809/809 [00:29<00:00, 27.71it/s]


Y (721, 1)
X (43200, 721)


# Initialize parameters

In [8]:
def initialze_params(dim):
    w = np.zeros((dim, 1))
    b = 0
    return w, b
w, b = initialze_params(dim)
print (w.shape, b)

(43200, 1) 0


# Forward Prop. and Calculating Grads.

In [9]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def propagate(w, b, X, Y):
    m = float(X.shape[1])
    A = sigmoid(np.dot(w.T, X) + b)  # compute activation
    A = A.T
    loss = Y * np.log(A) + (1 - Y) * np.log(1 - A)
    sumL = np.sum(loss)
    cost = (-1 / m) * np.sum((Y * np.log(A) + (1 - Y) * np.log(1 - A)))  # compute cost
    
    dw = 1 / m * np.dot(X, (A - Y))
    db = 1 / m * np.sum(A - Y)

    assert (dw.shape == w.shape)
    assert (db.dtype == float)
    cost = np.squeeze(cost)
    assert (cost.shape == ())

    grads = {"dw": dw,
             "db": db}
    return grads, cost
grads, cost = propagate(w, b, X, Y)
print (grads)
print ('Cost: '+ str(cost))

{'dw': array([[0.        ],
       [0.        ],
       [0.        ],
       ...,
       [0.00069348],
       [0.00069348],
       [0.00069348]]), 'db': 0.43481276005547853}
Cost: 0.6931471805599453


# Gradient Descent

In [10]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost):
    costs = []
    for i in range(num_iterations):
        grads, cost = propagate(w, b, X, Y)
        dw = grads["dw"]
        db = grads["db"]
        w = w - learning_rate * dw
        b = b - learning_rate * db

        # Record the costs
        if i % 100 == 0:
            costs.append(cost)

        # Print the cost every 100 training iterations
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" % (i, cost))
    print ('Cost after iteration {num_iterations}:'.format(num_iterations=num_iterations), cost)
    params = {"w": w,
              "b": b}
    grads = {"dw": dw,
             "db": db}

    return params, grads, costs
params, grads, costs = optimize(w, b, X, Y, num_iterations = 1000, learning_rate=0.009, print_cost=True)
print (grads)

Cost after iteration 0: 0.693147
Cost after iteration 100: 0.191660
Cost after iteration 200: 0.157649
Cost after iteration 300: 0.133540
Cost after iteration 400: 0.115336
Cost after iteration 500: 0.101073
Cost after iteration 600: 0.089606
Cost after iteration 700: 0.080203
Cost after iteration 800: 0.072374
Cost after iteration 900: 0.065774
Cost after iteration 1000: 0.06020196957811653
{'dw': array([[0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       ...,
       [7.52786163e-06],
       [7.52786163e-06],
       [7.52786163e-06]]), 'db': 0.003084592137447591}


In [11]:
def predict(pokemon_name):
    img_to_predict = img_to_lst('/kaggle/input/pokemon-images-and-types/images/images/' + pokemon_name)
    A_pred = sigmoid(np.dot(params['w'].T, img_to_predict) + params['b'])
    return A_pred

list_element_count = 0
img_track = 0
for x in os.listdir('/kaggle/input/pokemon-images-and-types/images/images'):
    temp = None
    temp = x.split('.')
    if temp[1] == 'png':
        print (x, predict(x), df.iloc[list_element_count]['Type1'])
        img_track += 1
    else:
        print (x, 'png skipped! ', df.iloc[list_element_count]['Type1'])
    list_element_count += 1
    
#accuracy = 

throh.png [0.00196967] Grass
samurott.png [0.02232976] Grass
venomoth.png [0.02135476] Grass
latios.png [0.7028448] Fire
shinx.png [0.59638776] Fire
durant.png [0.6578102] Fire
geodude.png [0.0358261] Water
wingull.png [0.06180043] Water
miltank.png [0.01792169] Water
castform.png [0.01251316] Bug
squirtle.png [0.03742515] Bug
shaymin-land.png [0.02518748] Bug
raikou.png [0.00035508] Bug
mawile.png [0.07050605] Bug
gallade.png [0.00262082] Bug
type-null.jpg png skipped!  Normal
patrat.png [0.00927524] Normal
froslass.png [0.03212194] Normal
lurantis.jpg png skipped!  Normal
nidoking.png [0.00317669] Normal
magnemite.png [0.02350547] Normal
cloyster.png [0.06258819] Normal
spinarak.png [0.01499138] Poison
meloetta-aria.png [0.03493018] Poison
melmetal.jpg png skipped!  Electric
exploud.png [0.00723573] Electric
vanillish.png [0.01121692] Ground
scrafty.png [0.05958089] Ground
azelf.png [0.00381039] Poison
mew.png [0.02904989] Poison
dialga.png [0.00288298] Poison
cryogonal.png [0.000179